# GOLUB REINSCH SVD

In [64]:
import numpy as np

In [65]:
def Set_Lowval_Zero(M):
    # This is for turning values into zeroes once they reach a certain threshold
    low_values_indices = abs(M) < 1e-5
    M[low_values_indices] = 0
    
    return M

In [66]:
def Householder_Reflector(x, i):
    # For computing x, v, and w vectors as well as the P matrix
    # We change the sign depending on the sign of first element of the x vector
    a = -np.sign(x[i]) * np.linalg.norm(x)  
    e = np.zeros(len(x))
    e[i] = 1.0

    # We then calculate the v and w vector as well as the P matrix
    w = (x - a * e)
    v = w / np.linalg.norm(w)
    P = np.eye(len(x)) - 2 * np.outer(v, v.T)

    return P, x, w, v

In [67]:
def Golub_Kahan_Bidiagonalization(A):
    m, n = A.shape
    # Initialize U and V as identity matrices
    U = np.eye(m)
    V = np.eye(n)

    # This algorithms will only run householder reflectors in the minimum no. of rows and columns
    # This is for cases of a non-square matrix A
    # Excess rows or columns will be left out
    if m <= n:
        num_iter = m - 1
    else:
        num_iter = n - 1

    for i in range(num_iter):
        # Performing Householder Reflectors column wise
        x = np.zeros(len(A[:, i]))
        x[i:] = A[i:, i]
        P, x, w, v = Householder_Reflector(x, i)
        A = Set_Lowval_Zero(P @ A)
        U = U @ P  # Update matrix U

        # Performing Householder Reflectors row wise
        x = np.zeros(len(A[i, :]))
        x[i+1:] = A[i, i+1:] 
        Q, x, w, v = Householder_Reflector(x, i+1)
        A = Set_Lowval_Zero(A @ Q)
        V = Q @ V  # Update matrix V

    print(A)
    return A, U, V

In [93]:
def Apply_Givens_Rotation(a,b):
    print("a: ", a)
    print("b: ", b)
    r = np.sqrt(abs(a)**2 + abs(b)**2)
    print("r: ", r)

    cos = a/r
    sin = b/r
        
    G = np.eye(2)
    G[0,0] = G[1,1] = cos
    G[0,1] = -sin
    G[1,0] = sin
    print("G: \n", G)

    return G

In [69]:
def Golub_Reinsch_SVD(A, max_iter = 1000, eps = 1e-5):
    m, n = A.shape
    B, U, V = Golub_Kahan_Bidiagonalization(A)
    
    
    if m < n:
        transposed = True
        m, n = n, m
        B = B.T
        U, V = V, U
    else:
        transposed = False
    
    Q = np.eye(m)
    P = np.eye(n)
    print(B)
    
    # Zero out the superdiagonals
    for i in range(n-1):
        if abs(B[i,i+1]) <= eps*(abs(B[i,i]) + abs(B[i+1,i+1])):
            B[i, i+1] = 0
    
    i = 0
    while True:
        # Find largest q
        for i in range(n-1, 0, -1):
            if i == n-1:
                if abs(B[i-1, i]) > 0:
                    q = 0
                    break
            
            else:
                if abs(B[i-1, i]) > eps or abs(B[i+1, i]):
                    q = n-i-1
                    break
        
            if i == 1 and np.all(np.array(np.abs([B[1,0],B[1,2],B[0,1]])) < eps):
                q = n
        
        if q == n:
            print("Matrix converged")
            Sigma = B
            break
            
        
        B_super = np.diag(B, 1)
        print("B_super: ", B_super)
        p = min(np.where(np.abs(B_super) > eps)[0])
        print("p: ", p)
        print("matrix B before: \n", B)
        
        for i in range(p, n-q-1):
            if abs(B[i,i]) < eps:
                G = Apply_Givens_Rotation(0, B[i,i+1])
                G = G.T
                
                R = np.eye(n)
                R[i:i+2, i:i+2] = G
                
                B, P = Multiply_Givens_Rotation(B, P, R, "Right")
                print("Givens")
                print("matrix B after: \n", B)
            else:
                B, Q, P = Golub_Kahan_SVD_step(n, B, Q, P, p, q)
                print("GolubKahan")
                print(B)
                print("matrix B after: \n", B)
        i += 1

    Sigma = B
    V = V @ P
    U = U @ Q
    
    if transposed:
        return V, Sigma, U
    else:
        return U, Sigma, V
        

In [70]:
def Schur_Decomposition(M):
    a, b, c, d = M[0,0], M[0,1], M[1,0], M[1,1]

    # Compute trace and determinant
    trace = a + d
    det = a * d - b * c

    # Calculate eigenvalues
    discriminant = (trace ** 2 - 4 * det) ** 0.5
    l1 = (trace + discriminant) / 2
    l2 = (trace - discriminant) / 2

    return l1, l2

In [99]:
def Multiply_Givens_Rotation(B, Q, P, R, side, i):
    if side == "Right":       
        # Multiply matrix B and matrix P by R_ij(c, s) on the right
        B = B @ R  # Right-multiplication of B_matrix by R_ij(c, s)
        B = Set_Lowval_Zero(B)
        P = P @ R  # Right-multiplication of P_matrix by R_ij(c, s)

        return B, P

    elif side == "Left":
        # Multiply matrix B and matrix Q by R_ij(c, -s) on the left
        B = R @ B  # Left-multiplication of R_ij(c, -s) by matrix B
        B = Set_Lowval_Zero(B)
        Q = Q @ R.T  # Right-multiplication of matrix Q by R_ij(c, -s)
        
        return B, Q
    
    elif side == "QR":
        # Multiply Matrix B and matrix Q by R_ij(c, -s) on the left 
        Sh = np.copy(P)
        G = np.copy(B)
        R[i:i+2, i:i+2] = G @ Sh
        R = Set_Lowval_Zero(R)
        print("Q: ", Q)
        Q[:, i:i+2] = Q[:, i:i+2] @ G.T
        
        return Q, R
        
       

In [72]:
def Golub_Kahan_SVD_step(n, B, Q, P, p, q):
    m = B.shape[0]
    B_2 = B[p:n-q, p:n-q]
    '''
    C = (B_2.T @ B_2)
    i,j = C.shape
    C = C[i-2:,j-2:]
    l1, l2 = Schur_Decomposition(C)
    diff1, diff2 = abs(l1 - C[1,1]), abs(l2 - C[1,1])

    if diff1 < diff2:
        Mu = diff1
    else:
        Mu = diff2 
    '''    
    k = p
    alpha = B[k, k]
    beta = B[k, k+1]
    for k in range(p, n-q-1):
        
        if alpha != 0 and beta != 0:
            # Determine c and s
            G = Apply_Givens_Rotation(alpha,beta)
            print("G matrix: \n", G)

            R = np.eye(n)
            R[k:k+2, k:k+2] = G
            print("R matrix: \n", R)

            # Update B_matrix and P_matrix with right multiplication
            print("Right")
            B, P = Multiply_Givens_Rotation(B, Q, P, R, "Right", k)
            print("B: \n", B)

        # Update alpha and beta
        alpha = B[k, k]
        beta = B[k+1, k]
        
        if alpha != 0 and beta != 0:
            # Determine c and s
            G = Apply_Givens_Rotation(alpha,beta)
            G = G.T
            print("G matrix: \n", G)

            R = np.eye(m)
            R[k:k+2, k:k+2] = G
            print("R matrix: \n", R)

            # Update B_matrix and Q_matrix with left multiplication
            print("Left")
            B, Q = Multiply_Givens_Rotation(B, Q, P, R, "Left", k)
            print("B: \n", B)

        if k <= n-q-3:
            alpha = B[k, k+1]
            beta = B[k, k+2]

    return B, P, Q

In [73]:
#We use np.random to generate a random matrix
A = np.random.rand(4, 4)

print("A matrix = \n\n", A)

U, Sigma, V = Golub_Reinsch_SVD(A)
print("U matrix = \n\n", U)
print("Sigma = \n\n", Sigma)
print("V matrix = \n\n", V)

A matrix = 

 [[0.38903247 0.28124576 0.01792064 0.72620012]
 [0.25539975 0.14253501 0.59567607 0.64527547]
 [0.30459049 0.56504875 0.07171959 0.12628737]
 [0.5871242  0.17027673 0.8133849  0.7151508 ]]
[[-0.80874316  1.47785128  0.          0.        ]
 [ 0.          0.46218139  0.39825246  0.        ]
 [ 0.          0.         -0.44778662  0.12294705]
 [ 0.          0.          0.          0.41088417]]
[[-0.80874316  1.47785128  0.          0.        ]
 [ 0.          0.46218139  0.39825246  0.        ]
 [ 0.          0.         -0.44778662  0.12294705]
 [ 0.          0.          0.          0.41088417]]
B_super:  [1.47785128 0.39825246 0.12294705]
p:  0
matrix B before: 
 [[-0.80874316  1.47785128  0.          0.        ]
 [ 0.          0.46218139  0.39825246  0.        ]
 [ 0.          0.         -0.44778662  0.12294705]
 [ 0.          0.          0.          0.41088417]]
a:  -0.8087431552773521
b:  1.4778512805953525
r:  1.6846690769303303
G matrix: 
 [[-0.48006055 -0.87723536]
 [ 

# GOLUB KAHAN TRIDIAGONALIZED SVD

In [105]:
def Transform_to_square(B):
    # If in case the resulting bidiagonalized matrix is not square
    m, n = B.shape

    # If there num of rows is greater than or equal to num of columns
    if m >= n:
        C = B[0:n, 0:n]
        dim = n
    else:
        # If num columns > num rows, get the number of excess columns
        add_cols = n - m
        if add_cols > 0:
            add_rows = np.zeros((add_cols, m + add_cols))
        else:
            add_rows = np.zeros((m + add_cols))
        C = np.vstack((B[:m, :m + n], add_rows))
        print(C)
        dim = m + add_cols

    return C, dim


In [106]:
def Golub_Kahan_Tridiagonalization(A):

    B, U, V = Golub_Kahan_Bidiagonalization(A)
    # Reshape bidiagonalized matrix into a square
    B, dim = Transform_to_square(B)

    # Create matrix O full of zeroes
    O = np.zeros((dim, dim))

    # Create matrix M with blocking of O and B matrices
    M = np.block([[O, B.T],[B, O]])
    M_rows, M_cols = M.shape

    # Create Permutation Matrix
    P = np.zeros((M_cols, M_cols))

    # The algorithm for rearrnging Permutation matrix
    for i in range(dim):
        # Build permutation matrix depending on the dimensions of Matrix B
        P[i, i*2] = 1
        P[dim+i,2*i + 1] = 1

    # Computing for the tridiagonalized matrix by performing PᵀMP
    Tri = P.T @ M @ P
    
    print(Tri)

    return Tri, U, V


In [107]:
def Wilkinson_Shift(A):
    
    # Compute for the shift 
    A_2x2 = A[-2:, -2:] # Bottom most right of matrix A [(a,b),(b,c)]. We expect it to be symmetric but non-symmetric should be fine
    a = A_2x2[0,0]
    b = A_2x2[0,1]
    c = A_2x2[1,0]

    print("")
    print("\033[1mBottom most right 2x2 of matrix A:\033[0m")
    print("")
    print(A_2x2)

    l = (a-c)/2

    if l < 0:
        shift = c + (b*b)/(abs(l)+(l*l + b*b)**0.5)
    elif l > 0:
        shift = c - (b*b)/(abs(l)+(l*l + b*b)**0.5)
    else:
        shift = c + (b*b)/(abs(l)+(l*l + b*b)**0.5)
    
    return shift


In [108]:
def QR_algorithm_for_SVD(A, max_iter = 1000, tol = 1e-6):
    
    B, U, V = Golub_Kahan_Tridiagonalization(A)
    n = B.shape[0] 

    Q = np.eye(n)
    P = np.eye(n)
    R = np.copy(B)
    
    U = np.eye(B.shape[0])  # Initialize U
    V = np.eye(B.shape[1])  # Initialize V

    # Initialize Variables
    eigenvalues = np.zeros(n, dtype=float)
    eigenvectors = np.identity(n, dtype=float)
    shift = 0.0  # Initial guess for the eigenvalue 


    for i in range(max_iter):

        # Compute for the shift 
        shift = Wilkinson_Shift(R)
        print("shift: ", shift)

        # Compute QR decomposition of Tridiagonalized matrix
        for i in range(n - 1):
            R_2x2 = R[i:i+2, i:i+2]
            Sh_M = np.eye(2) * shift
            Sh = R_2x2 - Sh_M
            a = Sh[0, 0]
            b = Sh[1, 0]
            G = Apply_Givens_Rotation(a,b)
            R[i:i+2, i:i+2] = G
            print(R)
            Q, R = Multiply_Givens_Rotation(G, Q, Sh, R, "QR", i)
            
            # Update B
            B = np.dot(R, Q)
            
            # Step 3: Update U and V
            U = np.dot(U, Q)
            V = np.dot(V, R.T)

        # Check for convergence
        eigenvalues_new = np.diag(B)

        if np.all(np.abs(eigenvalues - eigenvalues_new) < tol):
            break

        eigenvalues = eigenvalues_new
    
    if i == max_iter - 1:
        print("")
        print("\033[1mQR algorithm did not converge to the desired tolerance.\033[0m")
        print("\033[1mIterations: \033[0m" + str(i + 1))

    # Find Eigenvalues of B
    eigenvalues = B
    Sigma = eigenvalues
    

    if np.allclose(B, B.T, rtol=tol):
        eigenvectors = np.linalg.inv(eigenvectors)
        return Sigma, eigenvectors, U, V
    else:
        return Sigma, None, U, V


In [109]:
A = np.random.rand(2,2)

print("")
print("\033[1minput matrix A:\033[0m")
print("")
print(A)

QR_algorithm_for_SVD(A)


input matrix A:

[[0.31252419 0.56901249]
 [0.43443495 0.60722684]]
[[-0.53516829  0.82521842]
 [ 0.          0.10730427]]
[[ 0.         -0.53516829  0.          0.        ]
 [-0.53516829  0.          0.82521842  0.        ]
 [ 0.          0.82521842  0.          0.10730427]
 [ 0.          0.          0.10730427  0.        ]]

Bottom most right 2x2 of matrix A:

[[0.         0.10730427]
 [0.10730427 0.        ]]
shift:  0.17362195066441438
a:  -0.17362195066441438
b:  -0.5351682889047463
r:  0.5626274781787238
G: 
 [[-0.30859131  0.95119472]
 [-0.95119472 -0.30859131]]
[[-0.30859131  0.95119472  0.          0.        ]
 [-0.95119472 -0.30859131  0.82521842  0.        ]
 [ 0.          0.82521842  0.          0.10730427]
 [ 0.          0.          0.10730427  0.        ]]
Q:  [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
a:  0.38900552751430945
b:  0.8252184249258232
r:  0.9123106648909365
G: 
 [[ 0.4263959  -0.90453664]
 [ 0.90453664  0.4263959 ]]
[[-0.45547103  0.      

[[ 9.69119895e-01 -2.46590003e-01  0.00000000e+00  0.00000000e+00]
 [ 2.46590003e-01  9.69119895e-01  2.51156904e+07  0.00000000e+00]
 [ 0.00000000e+00 -3.26470044e+07  9.25012964e+07  2.68107992e+07]
 [ 0.00000000e+00  0.00000000e+00 -5.08913382e+07  4.88426633e+07]]
Q:  [[ 0.53472596 -0.67019777 -0.06198774  0.51094092]
 [ 0.39984538 -0.14279624 -0.59948285 -0.67849334]
 [-0.73584247 -0.62256457 -0.25485844 -0.07743638]
 [ 0.11281703 -0.377965    0.75619112 -0.5221013 ]]
a:  192857222.55032337
b:  -32647004.431105636
r:  195600959.06756076
G: 
 [[ 0.98597279  0.16690616]
 [-0.16690616  0.98597279]]
[[ 9.49015818e+07 -7.47568956e+07  0.00000000e+00  0.00000000e+00]
 [ 5.16381470e+07  9.85972786e-01  1.66906157e-01  0.00000000e+00]
 [ 0.00000000e+00 -1.66906157e-01  9.85972786e-01  2.68107992e+07]
 [ 0.00000000e+00  0.00000000e+00 -5.08913382e+07  4.88426633e+07]]
Q:  [[ 0.68347763 -0.51764391 -0.06198774  0.51094092]
 [ 0.42271024 -0.0397888  -0.59948285 -0.67849334]
 [-0.55960138 -0.

a:  3.772758518233722e+16
b:  -1.0245817905424038e+16
r:  3.909408482419838e+16
G: 
 [[ 0.96504587  0.26208103]
 [-0.26208103  0.96504587]]
[[ 1.91923844e+16 -1.49258842e+16  0.00000000e+00  0.00000000e+00]
 [ 1.08776155e+16  3.43529931e+16  1.38376602e+16  0.00000000e+00]
 [ 0.00000000e+00 -1.80495451e+16  9.65045872e-01  2.62081027e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.62081027e-01  9.65045872e-01]]
Q:  [[-0.16636973 -0.71018106 -0.64713797 -0.22175758]
 [-0.16312918 -0.18317987  0.52273726 -0.81644336]
 [-0.84327276  0.46197142 -0.2561929  -0.09919003]
 [-0.48435632 -0.49866864  0.49226354  0.52383695]]

Bottom most right 2x2 of matrix A:

[[ 3.37236159e+16  1.11009523e+16]
 [-1.97753686e+16  1.89683375e+16]]
shift:  -2.198734171073062e+16
a:  4.117972608590314e+16
b:  1.0877615483261542e+16
r:  4.25921631184858e+16
G: 
 [[ 0.9668381  -0.25539007]
 [ 0.25539007  0.9668381 ]]
[[ 9.66838101e-01 -2.55390069e-01  0.00000000e+00  0.00000000e+00]
 [ 2.55390069e-01  9.66838101e-01  1.3

a:  1.825785477191695e+26
b:  -5.03170141090315e+25
r:  1.8938513140183294e+26
G: 
 [[ 0.96405957  0.26568619]
 [-0.26568619  0.96405957]]
[[ 9.40766315e+25 -7.50851995e+25  0.00000000e+00  0.00000000e+00]
 [ 5.61356104e+25  9.64059567e-01  2.65686191e-01  0.00000000e+00]
 [ 0.00000000e+00 -2.65686191e-01  9.64059567e-01  2.92928811e+25]
 [ 0.00000000e+00  0.00000000e+00 -5.06903413e+25  4.86346386e+25]]
Q:  [[ 0.22253097 -0.83050458  0.47146685 -0.19611504]
 [-0.86737001 -0.04598823  0.12941339 -0.47833724]
 [-0.44487625 -0.34084    -0.04645918  0.82689465]
 [-0.01530856  0.43814987  0.87109857  0.22130886]]
a:  1.823111074815438e+26
b:  -5.069034128031226e+25
r:  1.8922698171841547e+26
G: 
 [[ 0.96345197  0.26788115]
 [-0.26788115  0.96345197]]
[[ 9.40766315e+25 -7.50851995e+25  0.00000000e+00  0.00000000e+00]
 [ 5.61356104e+25  1.62648060e+26  7.48647969e+25  0.00000000e+00]
 [ 0.00000000e+00 -9.70171976e+25  9.63451966e-01  2.67881149e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.67881

a:  3.255718560943751e+36
b:  -9.07378875794465e+35
r:  3.3797987768965887e+36
G: 
 [[ 0.96328769  0.26847127]
 [-0.26847127  0.96328769]]
[[ 1.67808766e+36 -1.35186486e+36  0.00000000e+00  0.00000000e+00]
 [ 1.01351183e+36  9.63287691e-01  2.68471272e-01  0.00000000e+00]
 [ 0.00000000e+00 -2.68471272e-01  9.63287691e-01  5.26358167e+35]
 [ 0.00000000e+00  0.00000000e+00 -9.07436331e+35  8.70705484e+35]]
Q:  [[-0.56443296  0.2017605  -0.37603968 -0.70661325]
 [-0.56571749 -0.38093017  0.72993229 -0.0453297 ]
 [-0.08946889  0.90163342  0.40813272  0.11171472]
 [-0.59445309  0.03524322 -0.39902468  0.69725372]]
a:  3.255048194154709e+36
b:  -9.07436331030893e+35
r:  3.3791684540940895e+36
G: 
 [[ 0.96326899  0.26853835]
 [-0.26853835  0.96326899]]
[[ 1.67808766e+36 -1.35186486e+36  0.00000000e+00  0.00000000e+00]
 [ 1.01351183e+36  2.89258846e+36  1.35226298e+36  0.00000000e+00]
 [ 0.00000000e+00 -1.74813380e+36  9.63268993e-01  2.68538353e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.685383

...


Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]

Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan



Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]

Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan


[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]

Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan n

a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]

Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
a:  nan
b:  nan
r:  nan
G: 
 [[nan nan]
 [nan nan]]
[[nan nan  0.  0.]
 [nan nan nan  0.]
 [ 0. nan nan nan]
 [ 0.  0. nan nan]]
Q:  [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]

Bottom most right 2x2 of matrix A:

[[nan nan]
 [nan nan]]
shift:  nan
a:  nan
b:  nan


(array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 None,
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan],
        [nan, nan, nan, nan]]))